In [1]:
import os
import sys
import time
import pickle
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.neural_network import MLPRegressor

# Add the parent directory to the Python path to load funtions from file ML_funtions
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
sys.path.append(parent_directory)

# Import helperfunctions
from ML_functions import fun_load_data, fun_preprocessing, fun_fit_tuning, fun_load_best_params
from ML_functions import fun_convert_time
from ML_functions import fun_scaled_neg_MAPE, fun_tuning_results, fun_scores

# Assign string 'TSP' or 'CVRP' to the following variable to define the optimization problem
optimization_problem = 'Bin_Packing'
train_size = 0.7

# Load data
data = fun_load_data(optimization_problem)
X, y, train_data = fun_preprocessing(data)

# Create a train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=42)

In [2]:
# The solver 'lbfgs' performed best, but was very slow. Therefore, these tests are done with the best parameters for the 'adam' solver.
best_params = {'mlpregressor__alpha': 0.05, 'mlpregressor__batch_size': 'auto', 
               'mlpregressor__hidden_layer_sizes': (100, 100), 'mlpregressor__solver': 'adam'} # Or 'mlpregressor__hidden_layer_sizes': (100)

# Create pipeline
pipe = make_pipeline(StandardScaler(), 
                     MLPRegressor(activation='relu', learning_rate='adaptive', 
                                  max_iter=1000, shuffle=True, random_state=0))
pipe.set_params(**best_params)

# Estimate model performance with cross-validation on the train set (scoring: MAPE and RMSE)
model_results_dict = fun_scores(pipe, X_train, y_train)

#model_results_dict = {'MAPE': 4.8439, 'RMSE': 0.0213}

CV MAPE (scaled) train data:  4.9524 %
CV RMSE (scaled) train data: 0.0219
CV computation time: 24s


# **Neural Network - Multi Layer Perceptron**
**Hyperparametertuning**

In [5]:
# Define a pipeline
pipe = make_pipeline(StandardScaler(),
                     MLPRegressor(activation='relu', learning_rate='adaptive', 
                                  max_iter=1000, shuffle=True, random_state=42))

# Define parameter grid
param_grid = {'mlpregressor__hidden_layer_sizes': [(100), (50, 50), (100, 100)],
              'mlpregressor__alpha': [0.05, 0.1, 0.5, 1],
              'mlpregressor__solver': ['sgd', 'adam', 'lbfgs'],
              'mlpregressor__batch_size': ['auto', 32, 64]}
 
# Grid search
grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=3, 
                           scoring=fun_scaled_neg_MAPE, verbose=True, n_jobs=-1)
fit_time = fun_fit_tuning(search_method=grid_search, X_train=X_train, y_train=y_train, file_name=optimization_problem + '_NN_GS')

# Estimate model performance with cross validation on the train set (scoring: MAPE and RMSE)
model_results_dict = fun_scores(model=grid_search, X_train=X_train, y_train=y_train)

# View grid search CV scores of all parameter combinations
results_df = fun_tuning_results(search_method=grid_search, search_space=param_grid)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
Tuning fit time: 5h, 2m
CV MAPE (scaled) train data:  4.3347 %

Best model / parameter combination:


{'mlpregressor__alpha': 0.5,
 'mlpregressor__batch_size': 'auto',
 'mlpregressor__hidden_layer_sizes': (100, 100),
 'mlpregressor__solver': 'lbfgs'}

Cross validation scores of different parameter combinations:


,param_mlpregressor__hidden_layer_sizes,param_mlpregressor__alpha,param_mlpregressor__solver,param_mlpregressor__batch_size,mean_test_score,converted_mean_fit_time
0,"(100, 100)",0.5,lbfgs,auto,-0.043347,"48m, 58s"
1,"(100, 100)",0.5,lbfgs,32,-0.043347,"49m, 15s"
2,"(100, 100)",0.5,lbfgs,64,-0.043347,"48m, 38s"
3,"(50, 50)",1,lbfgs,64,-0.043667,"18m, 42s"
4,"(50, 50)",1,lbfgs,32,-0.043667,"19m, 39s"
5,"(50, 50)",1,lbfgs,auto,-0.043667,"20m, 4s"
6,"(50, 50)",0.5,lbfgs,auto,-0.045358,"20m, 1s"
7,"(50, 50)",0.5,lbfgs,32,-0.045358,"19m, 5s"
8,"(50, 50)",0.5,lbfgs,64,-0.045358,"19m, 29s"
9,"(100, 100)",1,lbfgs,64,-0.045429,"25m, 6s"


,param_mlpregressor__hidden_layer_sizes,param_mlpregressor__alpha,param_mlpregressor__solver,param_mlpregressor__batch_size,mean_test_score,converted_mean_fit_time
60,"(50, 50)",0.05,adam,32,-0.066082,"5m, 46s"
61,"(50, 50)",0.1,adam,64,-0.066948,"2m, 9s"
62,100,1,sgd,auto,-0.067192,"5m, 0s"
63,"(100, 100)",0.5,sgd,auto,-0.067875,"12m, 53s"
64,"(100, 100)",0.05,adam,32,-0.068011,"14m, 52s"
65,100,0.05,adam,32,-0.068579,"2m, 54s"
66,"(50, 50)",0.5,sgd,auto,-0.068950,"7m, 3s"
67,100,0.1,adam,32,-0.070272,"2m, 12s"
68,"(100, 100)",0.5,adam,auto,-0.071688,"1m, 8s"
69,"(50, 50)",0.5,adam,auto,-0.072189,41s


**Test Score**

In [10]:
# Load best parameters of the model
best_params = fun_load_best_params(optimization_problem + '_NN_GS_best_params.pkl')

#best_params = {'mlpregressor__alpha': 0.5, 'mlpregressor__batch_size': 'auto', 
#               'mlpregressor__hidden_layer_sizes': (100, 100), 'mlpregressor__solver': 'lbfgs'}

# Create pipeline
pipe = make_pipeline(StandardScaler(), 
                     MLPRegressor(activation='relu', learning_rate='adaptive', 
                                  max_iter=1000, shuffle=True, random_state=42))
pipe.set_params(**best_params)

# Estimate model performance with cross-validation on the train set and get scores on test set (scoring: MAPE and RMSE)
model_results_dict = fun_scores(model=pipe, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, compute_test_scores=True)

{'mlpregressor__alpha': 0.5,
 'mlpregressor__batch_size': 'auto',
 'mlpregressor__hidden_layer_sizes': (100, 100),
 'mlpregressor__solver': 'lbfgs'}

CV MAPE (scaled) train data:  4.3347 %
CV RMSE (scaled) train data: 0.0185
CV computation time: 19m, 25s

MAPE (scaled) test data:  4.4662 %
RMSE (scaled) test data: 0.0194
Model fit time: 8m, 13s

MAPE and RMSE on test data per instance size:


Number Items,7,8,9,10,11,12,13,14,15,Mean
MAPE,5.1155,4.8784,4.8988,4.6509,4.360,4.2968,4.2722,4.1237,4.2646,4.4662
RMSE,0.0228,0.0214,0.0213,0.0203,0.019,0.0186,0.0189,0.0173,0.0182,0.0194
